### Building Dataset

In [4]:
import os
import pandas as pd
#os.environ['TRANSFORMERS_CACHE']= ''

Loading dataset from the hub

In [5]:
from datasets import load_dataset
dataset = load_dataset("ted_talks_iwslt", language_pair=("en","es"), year="2014")

Using custom data configuration en_es_2014-55e54a2735c2ce8d
Reusing dataset ted_talks_iwslt (C:\Users\asr_l\.cache\huggingface\datasets\ted_talks_iwslt\en_es_2014-55e54a2735c2ce8d\1.1.0\43935b3fe470c753a023642e1f54b068c590847f9928bd3f2ec99f15702ad6a6)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 3068
    })
})

Keeping English and Spanish Ted subtitles

In [7]:
l_en = []
l_es = []
for ele in dataset['train']['translation']:
   l_en.append(ele['en'])
   l_es.append(ele['es'])

In [8]:
df = pd.DataFrame()
df['Original_Sentence'] = l_en
df['Translate_SP'] = l_es

In [9]:
df

,Original_Sentence,Translate_SP
0,We've made incredible advances in technology i...,Hemos hecho avances increíbles en tecnología e...
1,Krista Donaldson: The $80 prosthetic knee that...,Krista Donaldson: La prótesis de rodilla de US...
2,"Almost everything we own and use, at some poin...","Casi todo lo que tenemos y usamos, en algún mo..."
3,Rose George: Inside the secret shipping industry,Rose George: El transporte marítimo por dentro
4,Snow Dragon. Pure Imagination. Frozen Minotaur...,Dragón de Nieve. Pura Imaginación. Minotauro H...
...,...,...
3063,David Pogue: Simplicity sells,"David Pogue dice ""La Simplicidad Vende"""
3064,"In an emotionally charged talk, MacArthur-winn...","En una charla altamente emotiva, la activista ..."
3065,Majora Carter: Greening the ghetto,Un recorrido por la renovación urbana de la ma...
3066,Sir Ken Robinson makes an entertaining and pro...,Sir Ken Robinson plantea de manera entretenida...


Train/Validation/Test dataset split

In [10]:
import numpy as np
train, validation, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.8*len(df)), int(.9*len(df))])

In [11]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
import datasets
from datasets import Dataset, concatenate_datasets
dataset = ds.dataset(pa.Table.from_pandas(train).to_batches())
dataset = ds.dataset(pa.Table.from_pandas(validation).to_batches())
dataset = ds.dataset(pa.Table.from_pandas(test).to_batches())


### convert to Huggingface dataset
hg_dataset_train = Dataset(pa.Table.from_pandas(train))
hg_dataset_validation = Dataset(pa.Table.from_pandas(validation))
hg_dataset_test = Dataset(pa.Table.from_pandas(test))

In [12]:
complete_dict =  datasets.DatasetDict({"train":hg_dataset_train,"validation":hg_dataset_validation, "test":hg_dataset_test})

Dataset created

In [13]:
complete_dict

DatasetDict({
    train: Dataset({
        features: ['Original_Sentence', 'Translate_SP', '__index_level_0__'],
        num_rows: 2454
    })
    validation: Dataset({
        features: ['Original_Sentence', 'Translate_SP', '__index_level_0__'],
        num_rows: 307
    })
    test: Dataset({
        features: ['Original_Sentence', 'Translate_SP', '__index_level_0__'],
        num_rows: 307
    })
})

In [31]:
complete_dict.save_to_disk('ted_trans_dataset')